In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# Load model_path from config.py
from config import model_path

# Choose llama3 model path
llama3_path = model_path["llama3.1"]

# Load tokenizer and model using the "unsloth" approach if needed
# If you want to use Unsloth's AutoModelForCausalLM, make sure the pip package is installed and imported.
# Here is a generic loading example (you can substitute with Unsloth's classes if needed):

tokenizer = AutoTokenizer.from_pretrained(llama3_path)
model = AutoModelForCausalLM.from_pretrained(llama3_path)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [3]:
LLAMA3_WEBSHOP_SYSTEM_PROMPT = "You are an intelligent exploration agent that navigates through environments to accomplish tasks. Your goal is to analyze the current state, understand the task instruction, and determine the next action to take. Respond with only the action you want to execute, without any additional explanation or formatting."
instruction = "i am looking for x-large, red color women faux fur lined winter warm jacket coat, and price lower than 70.00 dollars"
state = {'url': 'http://127.0.0.1:3000/<session_id>', 'html': 'WebShop Search'}
available_actions = "[search]"
action_status = {'has_search_bar': True, 'clickables': ['search']}

prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{LLAMA3_WEBSHOP_SYSTEM_PROMPT} 
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Task Instruction: {instruction}

Current URL: {state['url']}

Current Webpage Display Text: {state['html']}

Available Actions: {available_actions}

Based on the current state and task instruction, what is the next action you should take?

If you want to search, response with "search[<search_query>]"
<|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>
"""

outputs = pipe(prompt, max_new_tokens=64)
full_text = outputs[0]["generated_text"]
assistant_response = extract_assistant_response(full_text)

print("完整输出:")
print(full_text)
print("\n" + "="*60)
print("提取的assistant回复:")
print(assistant_response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


完整输出:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an intelligent exploration agent that navigates through environments to accomplish tasks. Your goal is to analyze the current state, understand the task instruction, and determine the next action to take. Respond with only the action you want to execute, without any additional explanation or formatting. 
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Task Instruction: i am looking for x-large, red color women faux fur lined winter warm jacket coat, and price lower than 70.00 dollars

Current URL: http://127.0.0.1:3000/<session_id>

Current Webpage Display Text: WebShop Search

Available Actions: [search]

Based on the current state and task instruction, what is the next action you should take?

If you want to search, response with "search[<search_query>]"
<|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>
search[x-large, red color women faux fur lined winter warm jacket coat]

提取的assistant回复:
search[x

In [5]:
from env_adaptors.webshopt_llama_atopter import WebshopLlamaAtopter
a = WebshopLlamaAtopter("A")

Additionally, Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/faiss/loader.py:28: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numpy.__version__) >= "1.19":
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/setuptools/_distutils/version.py:337: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to 

Products loaded.

Keys cleaned.

Attributes loaded.

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 94065.89it/s]

Loaded 6910 goals.



/home/xingkun/webshop/web_agent_site/envs/web_agent_text_env.py:212: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texts = self._parse_html(html).findAll(text=True)
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/gym/envs/registration.py:619: UserWarning: WARN: Env check failed with the following message: You must specify an observation space (cf gym.spaces) cf https://github.com/openai/gym/blob/master/gym/spaces/
You can set `disable_env_checker=True` to disable this check.
  logger.warn(


In [9]:
generated_prompt = a.get_next_action_prompt(instruction, state, action_status)

In [ ]:
outputs = pipe(generated_prompt, max_new_tokens=64)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an intelligent exploration agent that navigates through environments to accomplish tasks. Your goal is to analyze the current state, understand the task instruction, and determine the next action to take. Respond with only the action you want to execute, without any additional explanation or formatting. 
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Task Instruction: i am looking for x-large, red color women faux fur lined winter warm jacket coat, and price lower than 70.00 dollars

Current URL: http://127.0.0.1:3000/<session_id>

Current Webpage Display Text: WebShop Search

Available Actions: [search]

Based on the current state and task instruction, what is the next action you should take?

If you want to search, response with "search[<search_query>]"
If you want to click, response follow the format: "click[name of the clickable]"

<|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>
search[x-large red wo

In [ ]:
# 示例：只提取assistant回复（不显示完整输出）
outputs = pipe(generated_prompt, max_new_tokens=64)
full_text = outputs[0]["generated_text"]

# 方法1：只获取提取的回复
assistant_response = extract_assistant_response(full_text)
print("只显示提取的回复:")
print(assistant_response)

# 方法2：显示完整输出和提取的回复
# assistant_response = extract_assistant_response(full_text, show_full=True)


In [ ]:
LLAMA3_WEBSHOP_SYSTEM_PROMPT = "You are an intelligent exploration agent that navigates through environments to accomplish tasks. Your goal is to analyze the current state, understand the task instruction, and determine the next action to take. Respond with only the action you want to execute, without any additional explanation or formatting."
instruction = "i am looking for x-large, red color women faux fur lined winter warm jacket coat, and price lower than 70.00 dollars"
state = {'url': 'http://127.0.0.1:3000/<session_id>', 'html': 'WebShop Search'}
available_actions = "[search]"
{'has_search_bar': True, 'clickables': ['search']}

prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{LLAMA3_WEBSHOP_SYSTEM_PROMPT} 
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Task Instruction: {instruction}

Current URL: {state['url']}

Current Webpage Display Text: {state['html']}

Available Actions: {available_actions}

Based on the current state and task instruction, what is the next action you should take?

If you want to search, response with "search[<search_query>]"
<|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>
"""

outputs = pipe(prompt, max_new_tokens=64)
print(outputs[0]["generated_text"])

In [ ]:
LLAMA3_WEBSHOP_SYSTEM_PROMPT = "You are an intelligent exploration agent that navigates through environments to accomplish tasks. Your goal is to analyze the current state, understand the task instruction, and determine the next action to take. Respond with only the action you want to execute, without any additional explanation or formatting."
instruction = "i am looking for x-large, red color women faux fur lined winter warm jacket coat, and price lower than 70.00 dollars"
state = {'url': 'http://127.0.0.1:3000/search_results/<session_id>/x-large+red+women+faux+fur+lined+winter+warm+jacket+coat/1', 'html': "Back to Search Page 1 (Total results: 50) Next > B09KP78G37 Women Faux Fur Lined Jacket Coat Winter Warm Thick Fleece Outwear Trench Zipper Plus Size Long Sleeve Plush Overcoat $47.41 to $59.07 B07ZXBGDXF Women's Coat, FORUU Winter Faux Fur Fleece Outwear Warm Lapel Biker Motor Aviator Jacket $21.49 to $24.99 B09MSZ4VS8 Goldweather Women Winter Leggings Printed Warm Fleece Lined Yoga Pants Trousers Thermal High Waist Thicken Cashmere Tights (Navy, L) $20.99 B09GLVMLMS Anniou Electric Heated Jacket Adjustable Temp USB Heated Coat Winter Hooded Jacket Men Women $64.99 to $114.99 B09C8YNWWB Clarks Womens Suede Leather Slipper with Gore and Bungee JMH2213 - Warm Plush Faux Fur Lining - Indoor Outdoor House Slippers For Women $44.95 B09S3BN15C Mens Linen Shirt,Men's Striped Shirts Casual Short Sleeve Button Down Shirts Regular Fit Hawaiian Shirts Beach Tees Tops $3.78 to $11.38 B000J3HZXS Laguna Floating Planting Bag, Aquatic Pond Planter $19.99 B07DKGJR74 Amazon Brand - Daily Ritual Women's 100% Cotton Oversized Fit V-Neck Pullover Sweater $20.66 to $29.2 B098XT346Y Fjackets Real Lambskin Sherpa Jacket - Mens Leather Jacket $187.0 to $219.0 B0957XW92M Giantex Faux Fur Vanity Stool Chair, Round Footstool Ottoman with Metal Legs for Living Room, Fluffy Chair with Furry Padded Seat, Makeup Stool for Bedroom, Decorative Furniture Footrest White $52.99"}
available_actions = "['back to search', 'next >', 'b09kp78g37', 'b07zxbgdxf', 'b09msz4vs8', 'b09glvmlms', 'b09c8ynwwb', 'b09s3bn15c', 'b000j3hzxs', 'b07dkgjr74', 'b098xt346y', 'b0957xw92m']"

prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{LLAMA3_WEBSHOP_SYSTEM_PROMPT} 
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Task Instruction: {instruction}

Current URL: {state['url']}

Current Webpage Display Text: {state['html']}

Clickables: {available_actions}

Based on the current state and task instruction, what is the next action you should take?

If you want to search, response with "search[<search_query>]"
If you want to click, response follow the format: "click[name of the clickable]"
<|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>
"""

outputs = pipe(prompt, max_new_tokens=64)
print(outputs[0]["generated_text"])

In [ ]:
LLAMA3_WEBSHOP_SYSTEM_PROMPT = "You are an intelligent exploration agent that navigates through environments to accomplish tasks. Your goal is to analyze the current state, understand the task instruction, and determine the next action to take. Respond with only the action you want to execute, without any additional explanation or formatting."
instruction = "i am looking for x-large, red color women faux fur lined winter warm jacket coat, and price lower than 70.00 dollars"
state = {'url': 'http://127.0.0.1:3000/search_results/<session_id>/x-large+red+women+faux+fur+lined+winter+warm+jacket+coat/1', 'html': "Back to Search Page 1 (Total results: 50) Next > B09KP78G37 Women Faux Fur Lined Jacket Coat Winter Warm Thick Fleece Outwear Trench Zipper Plus Size Long Sleeve Plush Overcoat $47.41 to $59.07 B07ZXBGDXF Women's Coat, FORUU Winter Faux Fur Fleece Outwear Warm Lapel Biker Motor Aviator Jacket $21.49 to $24.99 B09MSZ4VS8 Goldweather Women Winter Leggings Printed Warm Fleece Lined Yoga Pants Trousers Thermal High Waist Thicken Cashmere Tights (Navy, L) $20.99 B09GLVMLMS Anniou Electric Heated Jacket Adjustable Temp USB Heated Coat Winter Hooded Jacket Men Women $64.99 to $114.99 B09C8YNWWB Clarks Womens Suede Leather Slipper with Gore and Bungee JMH2213 - Warm Plush Faux Fur Lining - Indoor Outdoor House Slippers For Women $44.95 B09S3BN15C Mens Linen Shirt,Men's Striped Shirts Casual Short Sleeve Button Down Shirts Regular Fit Hawaiian Shirts Beach Tees Tops $3.78 to $11.38 B000J3HZXS Laguna Floating Planting Bag, Aquatic Pond Planter $19.99 B07DKGJR74 Amazon Brand - Daily Ritual Women's 100% Cotton Oversized Fit V-Neck Pullover Sweater $20.66 to $29.2 B098XT346Y Fjackets Real Lambskin Sherpa Jacket - Mens Leather Jacket $187.0 to $219.0 B0957XW92M Giantex Faux Fur Vanity Stool Chair, Round Footstool Ottoman with Metal Legs for Living Room, Fluffy Chair with Furry Padded Seat, Makeup Stool for Bedroom, Decorative Furniture Footrest White $52.99"}
available_actions = "['back to search', 'next >', 'b09kp78g37', 'b07zxbgdxf', 'b09msz4vs8', 'b09glvmlms', 'b09c8ynwwb', 'b09s3bn15c', 'b000j3hzxs', 'b07dkgjr74', 'b098xt346y', 'b0957xw92m']"

prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{LLAMA3_WEBSHOP_SYSTEM_PROMPT} 
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Task Instruction: {instruction}

Current URL: {state['url']}

Current Webpage Display Text: {state['html']}

Available Actions: [click]

Clickables: {available_actions}

Based on the current state and task instruction, what is the next action you should take?

If you want to search, response with "search[<search_query>]"
If you want to click, response follow the format: "click[name of the clickable]"
<|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>
"""

outputs = pipe(prompt, max_new_tokens=64)
print(outputs[0]["generated_text"])

In [ ]:
from env_adaptors.webshopt_llama_atopter import WebshopLlamaAtopter
a = WebshopLlamaAtopter("A")